#**Python & Twitter: Aplicações em 140 caracteres**

Esse notebook foi escrito para você, visando servir como base para a elaboração de projetos usando o Twitter e sua API.

Aqui, deixamos uma colaboração em formato de código para servir de pontapé inicial para seus futuros trabalhos. Então, use sua criatividade e vá longe!

##**Coletando dados do Twitter**
**Atenção:** Nas contas com níveis de acesso "Essential" e "Elevated", a API permite a coleta de dados gerados nos últimos 7 dias. Para buscar tweets no acervo geral do Twitter é necessária uma conta com nível de acesso "Academic research".


**Instalando e importando a versão mais recente da tweepy**

A utilização de algumas funcionalidades da API v2 do Twitter requer a versão mais recente da biblioteca *tweepy*, por isso vamos instalar direto do GitHub oficial.

In [ ]:
!pip install git+https://github.com/tweepy/tweepy.git
import tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-v4pulzna
  Running command git clone -q https://github.com/tweepy/tweepy.git /tmp/pip-req-build-v4pulzna
     |████████████████████████████████| 62 kB 563 kB/s 
  Created wheel for tweepy: filename=tweepy-4.12.1-py3-none-any.whl size=101664 sha256=85c5a763d77cec5473059864b17e6f0e0972d3d85dda065e15351a37b0b5c757
  Stored in directory: /tmp/pip-ephem-wheel-cache-31xmbcan/wheels/b4/a5/5a/5074abdb9f4bd5bd0e22631a63fc41ae2fa71ad83780ea18d1
Successfully built tweepy
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


**Inserido as chaves e tokens**

In [ ]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""
bearer_token = ""

**Autenticando token**

In [ ]:
client = tweepy.Client(bearer_token)

###**Construindo os parâmteros de busca**
Aqui criamos a variável mais importante para a coleta dos dados. A chamaremos de consulta pois vem do inglês [query](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query).<br/>
Nela é possível adicionar palavras-chave e alguns outros filtros, como idioma e elementos presentes no tweet, por exemplo.

Além da consulta, outros filtros podem ser adicionados como parâmetros para conseguirmos dados específicos atrelados aos tweets, como o nome e @ do autor.

In [ ]:
#Filtros para consulta
consulta = "(resun UFS) lang:pt"

#Data limite da coleta (Formato ISO 8601/RFC 3339)
#Exemplo de data: ["2022-03-14T23:59:00.00z"]
#data_fim = ["AAAA-MM-DDTHH:MM:SS.ssz"]

#Número de resultados (máximo por consulta na conta "Elevated Access" -> 100 tweets)
num_resultados = 100

#Campos -> acessa os dados
fields = ["created_at", "author_id"]

#Expansões dos campos -> acessa os dados relacionados aos campos
exp = ["author_id"]

###**Coletando os tweets**

In [ ]:
response = client.search_recent_tweets(query= consulta, max_results = num_resultados, tweet_fields = fields, expansions = exp)

#Coletando os dados dos usuários (@ e nome)
users = {u["id"]: u for u in response.includes["users"]}

#Armazenando os dados
tweets = response.data

###**Exibindo os tweets**

In [ ]:
for tweet in tweets:
    if users[tweet.author_id]:
        user = users[tweet.author_id]
        #Data de criação | @ do usuário | Nome do usuário | Tweet
        print(tweet.created_at, user.username, user.name, tweet.text)

###**Salvando os tweets**
**Atenção:** Segundo os termos e condições de uso da API não é permitido armazenar publicamente os tweets coletados. Então, nada de deixar os textos dos tweets públicos por aí.

Nossa sugestão é armazenar as IDs dos tweets, por segurança.

In [ ]:
import pandas as pd

#Nome das colunas no dataframe
colunas = ["Data", "Usuário", "Nome", "Tweet"]

#Dados que serão adicionados nas linhas do dataframe
dados = []

#Adiciona os dados coletados na lista "dados"
for tweet in response.data:
    if users[tweet.author_id]:
        user = users[tweet.author_id]
        dados.append([tweet.created_at, user.username, user.name, tweet.text])

#Criação do data frame
data_frame = pd.DataFrame(dados, columns = colunas)

#Gerando .csv através do data frame
data_frame.to_csv("Dados-Twitter.csv", encoding="UTF-8")

##**Fechando com chave de ouro**
Agora, precisamos que vocês abram o Twitter e postem, com a **#VIIISEMAC**, algum feedback sobre esse mini curso. </br>
**OBS.: O perfil precisa estar aberto** </br>
*Não se preocupem, o @ de vocês não será exibido.*

Aqui a brincadeira irá utilizar a função de stream da biblioteca tweepy. </br>
Vamos buscar pela #VIIISEMAC e ver o que vocês estão postando em tempo real.

In [ ]:
streaming_client = tweepy.StreamingClient(bearer_token)

In [ ]:
class tweetPrinter(tweepy.StreamingClient):
    tweets = []

    def on_tweet(self, tweet):
        self.tweets.append(tweet)
        print(tweet.text)

In [ ]:
stream_tweet = tweetPrinter(bearer_token)

stream_tweet.add_rules(tweepy.StreamRule("#VIIISEMAC"))

stream_tweet.filter()

In [ ]:
import pandas as pd
colunas = ["Tweet"]
linhas = []

for tweet in stream_tweet.tweets:
    linhas.append(tweet.text)

df = pd.DataFrame(linhas, columns=colunas)
df


##**Referências**
* [**Repositório da apresentação (contém todos os materiais)**](https://github.com/TarcisioAraujo7/VIII_SEMAC)
* [**API do Twitter**](https://developer.twitter.com/en/docs/twitter-api/)
* [**Documentação tweepy**](https://docs.tweepy.org/en/stable/)
* [**Modelos de objetos**](https://docs.tweepy.org/en/stable/v1_models.html)
* [**Como construir uma consulta**](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)
* [**Bot Férias UFS (GitHub)**](https://github.com/TarcisioAraujo7/bot-twitter-ferias-UFS)
* [**twitter-toolbox (GitHub)**](https://github.com/tuliosg/twitter-toolbox)